In [2]:
using LinearAlgebra

# Task 1

Given $A\in \mathbb{R}^{n\times n}$ and $x, b\in \mathbb{R}^n$ Please derive the backward rule of $\mathcal{L} = \|Ax - b\|_2$ either using the chain rules or the perturbative approach (from the last lecture).

$$
\begin{split}
    \frac{\partial \mathcal{L}}{\partial x} 
    & = \frac{\partial \mathcal{L}}{\partial ||Ax - b||_2} \frac{\partial \sqrt{(Ax - b)^T (Ax - b)}}{\partial (Ax - b)} \frac{\partial (Ax - b)}{\partial x} \\
    & = 1 \times \frac{1}{2 ||Ax - b||_2} \times 2(Ax - b)^T \times A\\
    & = \frac{(Ax - b)^T \times A}{||Ax - b||_2}
\end{split}
$$

# Task 2.1
Text compression
Given a text to be compressed:

Compressed sensing (also known as compressive sensing, compressive sampling, or sparse sampling) is a signal processing technique for efficiently acquiring and reconstructing a signal, by finding solutions to underdetermined linear systems. This is based on the principle that, through optimization, the sparsity of a signal can be exploited to recover it from far fewer samples than required by the Nyquist–Shannon sampling theorem. There are two conditions under which recovery is possible. The first one is sparsity, which requires the signal to be sparse in some domain. The second one is incoherence, which is applied through the isometric property, which is sufficient for sparse signals.

Please
1. Analyse the frequency of each char
2. Create an optimal Huffman coding for each char
3. Encode the text and count the length of total coding (not including the deliminators).

In [16]:
text = "Compressed sensing (also known as compressive sensing, compressive sampling, or sparse sampling) is a signal processing technique for efficiently acquiring and reconstructing a signal, by finding solutions to underdetermined linear systems. This is based on the principle that, through optimization, the sparsity of a signal can be exploited to recover it from far fewer samples than required by the Nyquist–Shannon sampling theorem. There are two conditions under which recovery is possible. The first one is sparsity, which requires the signal to be sparse in some domain. The second one is incoherence, which is applied through the isometric property, which is sufficient for sparse signals."

"Compressed sensing (also known as compressive sensing, compressive sampling, or sparse sampling) is a signal processing technique for efficiently acquiring and reconstructing a signal, by finding solutions to underdetermined linear systems. This is based on the principle" ⋯ 155 bytes ⋯ " theorem. There are two conditions under which recovery is possible. The first one is sparsity, which requires the signal to be sparse in some domain. The second one is incoherence, which is applied through the isometric property, which is sufficient for sparse signals."

In [3]:
total_length = length(text)

694

In [4]:
frequency_count = Dict{Char, Int}()
for c in text
    if haskey(frequency_count, c)
        frequency_count[c] += 1
    else
        push!(frequency_count, c => 1)
    end    
end

In [10]:
symbols = ""
probs = Float64[]
for symbol in keys(frequency_count)
    symbols *= symbol
    push!(probs, frequency_count[symbol] / total_length)
end

In [11]:
struct Node{VT, PT}
    value::Union{VT,Nothing}
	prob::PT
    left::Union{Node{VT,PT}, Nothing}
    right::Union{Node{VT,PT}, Nothing}
end

using DataStructures

function huffman_tree(symbols, probs)
	isempty(symbols) && error("empty input!")
	# priority queue can keep the items ordered with log(# of items) effort.
	nodes = PriorityQueue(Base.Order.Forward,
		[Node(c, f, nothing, nothing)=>f for (c, f) in zip(symbols, probs)])
    while length(nodes) > 1
        left = dequeue!(nodes)
        right = dequeue!(nodes)
        parent = Node(nothing, left.prob + right.prob, left, right)
        enqueue!(nodes, parent=>left.prob + right.prob)
    end
	return dequeue!(nodes)
end

huffman_tree (generic function with 1 method)

In [12]:
ht = huffman_tree(symbols, probs)

Node{Char, Float64}(nothing, 1.0, Node{Char, Float64}(nothing, 0.40057636887608067, Node{Char, Float64}(nothing, 0.18587896253602304, Node{Char, Float64}(nothing, 0.09221902017291066, Node{Char, Float64}('a', 0.04610951008645533, nothing, nothing), Node{Char, Float64}(nothing, 0.04610951008645533, Node{Char, Float64}(nothing, 0.023054755043227664, Node{Char, Float64}(nothing, 0.011527377521613832, Node{Char, Float64}('v', 0.005763688760806916, nothing, nothing), Node{Char, Float64}(nothing, 0.005763688760806916, Node{Char, Float64}(nothing, 0.002881844380403458, Node{Char, Float64}('–', 0.001440922190201729, nothing, nothing), Node{Char, Float64}('C', 0.001440922190201729, nothing, nothing)), Node{Char, Float64}(nothing, 0.002881844380403458, Node{Char, Float64}('(', 0.001440922190201729, nothing, nothing), Node{Char, Float64}('S', 0.001440922190201729, nothing, nothing)))), Node{Char, Float64}(',', 0.011527377521613832, nothing, nothing)), Node{Char, Float64}('g', 0.023054755043227664

In [13]:
function decent!(tree::Node{VT}, prefix::String="", d::Dict = Dict{VT,String}()) where VT
	if tree.left === nothing # leaft
		d[tree.value] = prefix
	else   # non-leaf
		decent!(tree.left, prefix*"0", d)
		decent!(tree.right, prefix*"1", d)
	end
	return d
end

decent! (generic function with 3 methods)

In [14]:
ht_code = decent!(ht)

Dict{Char, String} with 35 entries:
  'n' => "1001"
  'f' => "110010"
  'w' => "1100111"
  'd' => "110110"
  'e' => "001"
  'o' => "0110"
  ')' => "1000000110"
  'C' => "000100101"
  '–' => "000100100"
  'h' => "11010"
  '(' => "000100110"
  'y' => "010111"
  't' => "0100"
  ',' => "000101"
  'r' => "0111"
  'k' => "1000000111"
  'q' => "1000001"
  'N' => "100000001"
  ' ' => "101"
  ⋮   => ⋮

In [15]:
code_length = 0

for ht_key in keys(ht_code)
    frequency = frequency_count[ht_key]
    length_ht = length(ht_code[ht_key])
    code_length += frequency * length_ht
end
println("the total length of the coded text is ", code_length, ", and the mean code length is ", code_length / total_length, ".")

the total length of the coded text is 2992, and the mean code length is 4.311239193083574.


# Task 2.2 Compressed Sensing

Go through the video clip [Compressed Sensing: When It Works](https://youtu.be/hmBTACBGWJs)

Please summarize this video clip, and explain when does compressed sensing work and when not.

The video introduced the mathematical formula of compressed sensing method and discussed when do it work.

The relation of the original singal $x$ and its corresponding in Fourier space $s$ can be given as
$$
x = \Psi s
$$
where $\Psi$ is the Fourier basis and we assume that $s$ is sparse.
Then compressed sensing can be regarded as solving the equation given by
$$
y = C \Psi s
$$
where $y$ is a reduced order measurement of the original signal and $C$ is a random matrix representing the measurement.
Beacuse of the requirement that $s$ is sparse, we have to minimize L1 norm of $s$.

There are two requirements for the method to work, which are directly shown by the formula given above.

First, the measurement $C$ have to be **random** and incoherent with the basis $Psi$, or we will only get information about limited frequency corresponding to only part of $s$.

Second, size of $y$ can not be too small, if there are K non-zero elements in $s$, then length of $y$ should be given by
$$
O(K \log{(n / K)})\;,
$$
where $n$ is length of $s$.